https://qiita.com/yuki_uchida/items/09fda4c5c608a9f53d2f#ツイートを文章ベクトルに変換する

https://nlp.ist.i.kyoto-u.ac.jp/index.php?ku_bert_japanese　BERT学習済みモデル

In [54]:
import pandas as pd
import re

In [55]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import numpy as np
model = BertModel.from_pretrained('/Users/hoteison/Desktop/DoMyResearch/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers')
bert_tokenizer = BertTokenizer("/Users/hoteison/Desktop/DoMyResearch/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt",
                               do_lower_case=False, do_basic_tokenize=False)
# 相対パスだとダメだったが、絶対パスなら行けた
# もっと重いファイルを選べば精度が上がるらしい

Some weights of the model checkpoint at /Users/hoteison/Desktop/DoMyResearch/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
# https://yag-ays.github.io/project/pytorch_bert_japanese/

from pyknp import Juman

class JumanTokenizer():
    def __init__(self):
        self.juman = Juman()

    def tokenize(self, text):
        result = self.juman.analysis(text)
        return [mrph.midasi for mrph in result.mrph_list()]

juman_tokenizer = JumanTokenizer()

In [59]:
def compute_vector(text, model, bert_tokenizer, juman_tokenizer):
    use_model = model
    tokens = juman_tokenizer.tokenize(text)
    bert_tokens = bert_tokenizer.tokenize(" ".join(tokens))
    ids = bert_tokenizer.convert_tokens_to_ids(
        ["[CLS]"] + bert_tokens[:126] + ["[SEP]"])
    tokens_tensor = torch.tensor(ids).reshape(1, -1)
    use_model.eval()
    with torch.no_grad():
        all_encoder_layers, _ = use_model(tokens_tensor)
    pooling_layer = -2
    print(all_encoder_layers[0])
    embedding = all_encoder_layers[0][pooling_layer].numpy()
    # embedding = all_encoder_layers[0].numpy()
    # return np.mean(embedding, axis=0)
    return embedding

In [ ]:
reviews_df = pd.read_csv("/Users/hoteison/Desktop/DoMyResearch/arranged_tripadvisor.csv", encoding='utf-8')
reviews_df["body"] = reviews_df["body"].astype(str) #一応文字列にしておく
reviews_df = reviews_df["body"]
## 文章ベクトル変換後にその結果を格納するための配列と、元のツイートを保存しておくための配列を宣言する
vectors = []
reviews = []
for review in reviews_df:
    review = re.sub('\n', " ", review)  #改行文字のの削除
    strip_review = re.sub(r'[︰-＠]', "", review)  #全角記号の削除
    # print(strip_review, len(strip_review))

    try:
        if len(strip_review) > 3: #単語数が少なすぎると適切なベクトルが得られない可能性があるため
            vector = compute_vector(
                strip_review, model, bert_tokenizer, juman_tokenizer)
            vectors.append(vector)
            reviews.append(review)

            print(1)
    except Exception as e:
        continue
## 文章ベクトルに変換したものをtsvにおく。(可視化ツールがtsvを要求してくるのでtsvにする)
pd.DataFrame(reviews).to_csv('./reviews_text.tsv', index=False, header=None)
pd.DataFrame(vectors).to_csv('./reviews_vector.tsv', sep='\t', index=False, header=None)

"""
l
l
l
l
l
l
l
l
l
l
l
l
l
l
l
"""

1212　エラーの理由はよくわかってない
・可能性としては、Tweetはテキストの最大長さが決まっていて、Trip Advisorは決まっていないこと。
・長さを調整する部分を見つけ出したいところだが、126の部分ではなかった。まだ考える必要がある。